# Ollama local LLM 
因語言模型的免費次數上限, 我們在這邊安裝一個本地的LLM

In [1]:
from langchain.llms import Ollama

chat = Ollama(model="openchat:latest")

/Users/babyching/Documents/personal/LLM/從LangChain接入ChatGPT到製作股票分析AI團隊/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from langchain.document_loaders import PyPDFLoader
loaders = [
    PyPDFLoader("docs/01.pdf"),
    PyPDFLoader("docs/02.pdf"),
    PyPDFLoader("docs/03.pdf"),
    PyPDFLoader("docs/04.pdf"),
]

docs = []

for loader in loaders:
    docs.extend(loader.load())


from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap=100,
    length_function = len,
    separators=["\n\n", '\n', ' ', '']
    )

splits = text_splitter.split_documents(docs)

from langchain_openai import OpenAIEmbeddings
import os
embeddings = OpenAIEmbeddings(
    base_url = os.environ["EMBEDDINGS_BASE_URL"]
)

from langchain.vectorstores import Chroma
persist_directory = "./db"
!rm -rf ./db
vectordb = Chroma.from_documents(
    documents = splits,
    embedding = embeddings,
    persist_directory = persist_directory
)

In [3]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(name="source", description="搜索的資訊來源於以下三個PDF文黨`docs/01.pdf`, `docs/03.pdf`, `docs/04.pdf`",type="string",),
    AttributeInfo(name="page", description="資訊來源的頁面",type="integer",)
]

document_content_description = "這裡存放的是關於香港特色旅遊勝地以及美食和特有文化"
retriever = SelfQueryRetriever.from_llm(
    llm=chat,
    vectorstore=vectordb,
    document_contents=document_content_description,
    metadata_field_info = metadata_field_info,
)

In [7]:
question = "介紹一下香港特色美食？"

docs = retriever.invoke(question, k=5)

for doc in docs:
    print(doc.metadata)

{'page': 6, 'source': 'docs/04.pdf'}
{'page': 3, 'source': 'docs/01.pdf'}
{'page': 7, 'source': 'docs/03.pdf'}
{'page': 7, 'source': 'docs/04.pdf'}


### Summary Chroma information

In [9]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(chat)

compression_retriever = ContextualCompressionRetriever(
    base_retriever = vectordb.as_retriever(),
    base_compressor= compressor,
)

In [13]:
question = "介紹一下西貢優美景色"

compressed_docs = compression_retriever.invoke(question)

def pretty_print_docs(docs):
    print(
        f"\n\n{'-' * 60}".join([f"\n\n第{i+1}個檢索:\n\n" + d.page_content for i, d in enumerate(docs)])
    )

pretty_print_docs(compressed_docs)

/Users/babyching/Documents/personal/LLM/從LangChain接入ChatGPT到製作股票分析AI團隊/.venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/babyching/Documents/personal/LLM/從LangChain接入ChatGPT到製作股票分析AI團隊/.venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/babyching/Documents/personal/LLM/從LangChain接入ChatGPT到製作股票分析AI團隊/.venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/babyching/Documents/personal/LLM/從LangChain接入ChatGPT到製作股票分析AI團隊/.venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser



第1個檢索:

介紹一下西貢優美景色
相片提供：TUGO CHENG
海灘及小島
感受腳底的柔軟白沙和清澈海水，環顧周圍趣怪
的野生動物，這樣的香港或許會讓你意想不到。
西貢隱身於香港僻靜的一隅，擁有純淨無污染的
海灘。沿著郊野公園遠足攀山，西貢的美景盡收
眼底；你也可以從西貢市中心的西貢公眾碼頭搭
乘街渡（即小型渡輪），闖進另一番天地。
另外，西貢還有大大小小的島嶼，數不清的幽靜景
點俯拾皆是，絕對值得花上一天仔細探索。

------------------------------------------------------------

第2個檢索:

位於新界最東端的西貢，既沒有櫛比鱗次的摩天大廈，也不見五光十色的霓虹燈，有「香港後花園」之稱。依山傍海的美景，悠閒的氛圍，讓這裡成為在地人的度假勝地。

------------------------------------------------------------

第3個檢索:

西貢擁有五花百門的美食，讓你品嚐到不同的異國料理，每一條街道各有味道，由街頭小吃到米其林星級體驗，應有盡有。


In [14]:
compression_retriever = ContextualCompressionRetriever(
    base_retriever = vectordb.as_retriever(search_type="mmr"), # 改變搜索方式
    base_compressor= compressor,
)

question = "香港哪裡有最好吃的蛋塔? 如果有,請提供該店舖的地址"

compressed_docs = compression_retriever.invoke(question)
pretty_print_docs(compressed_docs)

/Users/babyching/Documents/personal/LLM/從LangChain接入ChatGPT到製作股票分析AI團隊/.venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/babyching/Documents/personal/LLM/從LangChain接入ChatGPT到製作股票分析AI團隊/.venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/babyching/Documents/personal/LLM/從LangChain接入ChatGPT到製作股票分析AI團隊/.venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/babyching/Documents/personal/LLM/從LangChain接入ChatGPT到製作股票分析AI團隊/.venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser



第1個檢索:

恆發雞蛋仔開在九龍城的，靠著別出心裁的鮮果入餡功夫，吸引大批食客，周潤發、朱茵等明星也愛這一味。 "原味雞蛋仔" 蛋味香濃，但恆發最拿手的是夾心雞蛋仔，芋頭、紅豆、南瓜、紫薯、芝麻入餡都不算稀奇，最厲害的是號稱全香港首創的鮮果夾心，夾的還是新鮮的金枕頭榴槤果肉。

------------------------------------------------------------

第2個檢索:

好吃的蛋塔在泰昌餅家，地址中環擺花街35號。

------------------------------------------------------------

第3個檢索:

金樺餅店曾被選為香港最好吃的蛋塔也正在出爐。位置：太安樓地下A34D號舖，營業時間：08 ： 45～22 ： 00

------------------------------------------------------------

第4個檢索:

地址：香港銅鑼灣告士打道310號27樓 （香港柏寧鉑爾曼酒店）  電話：＋852-2839-3327
營業時間：（雞尾酒供應） 12 ： 00～凌晨00 ： 30 交通：港鐵銅鑼灣站 E 出口步行3分鐘
Skye
1.  必吃甜點 「港式奶茶焦糖燉蛋、焦糖脆餅」 不只
外型復古，奶味也很香濃。 （港幣100元／份）
2.  提供小酌靈感的星座調酒系列很受遊客歡迎。
（港幣120元／杯）
3.  若想點硬派調酒，也能點杯 「Negroni」 試試調
酒師手藝。 （港幣140元／杯）
